# Electricity market basics

In this notebook, we will look at the basics of energy market operation. We will not yet consider energy networks, but instead look at an energy market located in a single point (so that we do not need to worry about any physical constraints on energy flows). NB: this notebook is heavy on computation, so it might take a while to load the examples.

This is not an electrical engineering course, so we will conveniently going to ignore very short-term operational problems (e.g., dynamics, and other things that happen in electrical systems on very short timescales of milliseconds to minutes); we will also ignore the intricacies of our modern alternating current networks. These are interesting but out of scope here. We will also confine ourselves to wholesale markets, where generators sell electricity to wholesale consumers (i.e., utilities and large industrial consumers). Utilities sell the power they buy on to consumers in consumer markets. These are less interesting from the perspective of energy market analysis, as consumers generally sign long-term contracts to buy electricity at a fixed price per unit, and competition depends more on marketing and service levels than on economics. You would expect, however, prices in consumer markets to be equal to a long-term average of wholesale market prices (as these represent the utility’s input costs), plus a markup to cover the utility’s other costs and profits, plus taxes. In the UK, consumer prices are approximately twice as high as wholesale market prices, but they do correlate over a number of years.

## Economic dispatch

The diagram below shows a simple example of four periods of energy market operation. Energy demand in the periods is assumed to be perfectly inelastic (i.e., not sensitive to prices), which is a reasonable approximation in the short term. Each period lasts 1 hour, such that, if a generator generates at 1MW in a particular period, it has generated 1MWh (i.e. MWs and MWhs are equivalent). Demand is given by

Period 1 1 MWh
Period 2 2 MWh
Period 3 3 MWh
Period 4 4 MWh

There are three types of generation. We have 2.1GW of coal-fired generation capacity, which has a constant marginal fuel cost of 8£/MWh and 2GW of gasfired generation a cost of 10£/MWh, while wind generation has no cost but a limited availability. Because of varying wind conditions, in each period, only the following percentage of its installed capacity is available:

Period 1 0 %
Period 2 0.5 %
Period 3 0.35 %
Period 4 0.9 %

You can choose the amount of installed wind capacity. In addition to their fuel costs, generators have to pay a carbon price if you select a nonzero value. Coal-fired generators emit 0.1t/MWh, and gas-fired generators 0.05t/MWh. Carbon prices are per tonne and due on all carbon the generators emit. Wind generators have no carbon emissions.

The widget below shows the optimal generation levels and prices which are obtained when we look for the intersection of the demand and supply curves in each period. This is also the perfectly competitive equilibrium, as we know that, without externalities or other distortions, perfect competition gives us the optimal quantities and prices. In mathematical terms, since there is no consumer surplus here (as the demand curve is vertical), we can obtain this point by minimising the total cost of generation (generation times variable generation costs, summed over all generators and all time periods), subject to a constraint that specifies that the sum of generation has to meet demand in each time period. Generation decisions are called dispatch decisions, so the generation levels correspond to optimal dispatch (or economic dispatch). The supply curve shows the merit order, i.e., the order in which generators should be dispatched in an optimal solution.

Experiment with the economic dispatch model below. Do you understand why prices are what they are (Note: they are the cost of the most expensive unit that is still producing, i.e. the price at the intersection of the supply and demand curves)? What happens to prices if we add progressively more wind into the system? Do you understand why? (this is called the merit order effect, because wind changes the supply curve). Would you be happy if you were one of the first to invest in wind turbines?

What happens if you increase carbon prices? Can you see coal and gas change place in the merit order? Do carbon emissions actually change if you increase the price? Why/why not?

If you had some energy storage capacity, when would you charge/discharge this? Would you be happy with more renewables or higher carbon prices, or not?

In [1]:
import pypsa, numpy as np

# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        html.P("Select wind capacity:"),
        dcc.Slider(
            id="slider-wind-capacity",
            min=0,
            max=2000,
            step=1,
            value=500,
            marks={i: str(i) for i in range(0, 2001, 250)},
        ),
        html.P("Select wind availability:"),
        dcc.Slider(
            id="slider-wind-availability",
            min=0,
            max=1,
            step=0.01,
            value=0.75,
            marks={i: str(i) for i in range(0, 2, 1)},
        ),
        html.P("Select gas capacity:"),
        dcc.Slider(
            id="slider-gas-capacity",
            min=500,
            max=2000,
            step=1,
            value=500,
            marks={i: str(i) for i in range(500, 2001, 250)},
        ),
        html.P("Select coal capacity:"),
        dcc.Slider(
            id="slider-coal-capacity",
            min=500,
            max=2000,
            step=1,
            value=500,
            marks={i: str(i) for i in range(500, 2001, 250)},
        ),
        html.P("Select electricity demand:"),
        dcc.Slider(
            id="slider-electricity-demand",
            min=1000,
            max=4000,
            step=100,
            value=1000,
            marks={i: str(i) for i in range(0, 4001, 500)},
        )
    ]
)

@app.callback(
    Output("graph", "figure"),
    Input("slider-wind-capacity", "value"),
    Input("slider-wind-availability", "value"),
    Input("slider-gas-capacity", "value"),
    Input("slider-coal-capacity", "value"),
    Input("slider-electricity-demand", "value"),
)

def calc(wind_capacity, wind_availability, gas_capacity, coal_capacity, electricity_demand):

    # carbon emissions in tCO2/MWh
    carbon_emissions = {"Wind": 0, "Coal": 0.1, "Gas": 0.05}
    # operational costs in £/MWh
    operational_costs = {"Wind": 0, "Coal": 8, "Gas": 10}
    # marginal costs in £/MWh
    carbon_price = 1
    marginal_costs = {"Wind": carbon_emissions['Wind'] * carbon_price + operational_costs['Wind'],
                    "Coal": carbon_emissions['Coal'] * carbon_price + operational_costs['Coal'],
                    "Gas": carbon_emissions['Gas'] * carbon_price + operational_costs['Gas']}
    # power plant capacities (nominal powers in MW)
    power_plant_p_nom = {'Bus': {"Coal": coal_capacity, "Wind": wind_capacity, "Gas": gas_capacity}}
    # country electrical loads in MW (not necessarily realistic)
    loads = {"Bus": [electricity_demand]}

    network = pypsa.Network()
    # snapshots labelled by [0,1,2,3]
    country = "Bus"
    network.set_snapshots(range(1))
    network.add("Bus", country)

    # p_max_pu is variable for wind
    for tech in power_plant_p_nom[country]:
        network.add(
            "Generator",
            "{} {}".format(country, tech),
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
            p_max_pu=([wind_availability] if tech == "Wind" else 1),
        )

    # load which varies over the snapshots
    network.add(
        "Load",
        "{} load".format(country),
        bus=country,
        p_set=np.array([electricity_demand]),
    )

    network.optimize(solver_name='highs')

    def demand_supply_diagram(period):

        # minimum marginal cost unit between gas and coal
        marginal_costs_ = marginal_costs
        del marginal_costs_['Wind']
        min_type = min(marginal_costs_, key=marginal_costs_.get)
        max_type = max(marginal_costs_, key=marginal_costs_.get)

        x0 = 0
        y0 = 0

        x1 = wind_capacity * network.generators_t.p_max_pu['Bus Wind'][period]
        y1 = 0

        x2 = x1
        y2 = marginal_costs[min_type]

        x3 = x2 + power_plant_p_nom['Bus'][min_type]
        y3 = y2

        x4 = x3
        y4 = marginal_costs[max_type]

        x5 = x4 + power_plant_p_nom['Bus'][max_type]
        y5 = y4

        fig = go.Figure()

        # Add shapes
        fig.add_shape(type="line",
            x0=x0, y0=y0, x1=x1, y1=y1,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text="Wind")
        )
        fig.add_shape(type="line",
            x0=x1, y0=y1, x1=x2, y1=y2,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x2, y0=y2, x1=x3, y1=y3,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=min_type)
        )
        fig.add_shape(type="line",
            x0=x3, y0=y3, x1=x4, y1=y4,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x4, y0=y4, x1=x5, y1=y5,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=max_type)
        )
        fig.update_shapes(dict(xref='x', yref='y'))

        fig.add_vline(x=network.loads_t.p['Bus load'][period], line_color="green", annotation_text="Load", annotation_position="bottom right")

        fig.update_xaxes(range=[0, network.generators.p_nom.sum() + 10], title_text='Quantity')
        fig.update_yaxes(range=[0, y5 + 10], title_text='Price')
        fig.update_layout(showlegend=False, margin=dict(t=20, b=20))

        return fig

    fig = demand_supply_diagram(0)

    return fig

app.run_server(mode='inline', port=8062, jupyter_height=900)

## Carbon pricing

Spiel on carbon pricing.

In [2]:
import pypsa, numpy as np

# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        html.P("Select carbon price:"),
        dcc.Slider(
            id="slider-carbon-price",
            min=0,
            max=80,
            step=8,
            value=0,
            marks={i: str(i) for i in range(0, 81, 8)},
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Input("slider-carbon-price", "value")
)

def calc(carbon_price):

    # carbon emissions in tCO2/MWh
    carbon_emissions = {"Wind": 0, "Coal": 0.1, "Gas": 0.05}
    # operational costs in £/MWh
    operational_costs = {"Wind": 0, "Coal": 7, "Gas": 10}
    # marginal costs in £/MWh
    marginal_costs = {"Wind": carbon_emissions['Wind'] * carbon_price + operational_costs['Wind'],
                    "Coal": carbon_emissions['Coal'] * carbon_price + operational_costs['Coal'],
                    "Gas": carbon_emissions['Gas'] * carbon_price + operational_costs['Gas']}
    # power plant capacities (nominal powers in MW)
    wind_capacity = 500
    power_plant_p_nom = {'Bus': {"Coal": 1000, "Wind": 500, "Gas": 1000}}
    # country electrical loads in MW (not necessarily realistic)
    loads = {"Bus": [2000]}

    network = pypsa.Network()
    # snapshots labelled by [0,1,2,3]
    country = "Bus"
    network.set_snapshots(range(1))
    network.add("Bus", country)

    # p_max_pu is variable for wind
    for tech in power_plant_p_nom[country]:
        network.add(
            "Generator",
            "{} {}".format(country, tech),
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
            p_max_pu=([0.5] if tech == "Wind" else 1),
        )

    # load which varies over the snapshots
    network.add(
        "Load",
        "{} load".format(country),
        bus=country,
        p_set=np.array([2000]),
    )

    network.optimize(solver_name='highs')

    def demand_supply_diagram(period):

        # minimum marginal cost unit between gas and coal
        marginal_costs_ = marginal_costs
        del marginal_costs_['Wind']
        min_type = min(marginal_costs_, key=marginal_costs_.get)
        max_type = max(marginal_costs_, key=marginal_costs_.get)

        x0 = 0
        y0 = 0

        x1 = wind_capacity * network.generators_t.p_max_pu['Bus Wind'][period]
        y1 = 0

        x2 = x1
        y2 = marginal_costs[min_type]

        x3 = x2 + power_plant_p_nom['Bus'][min_type]
        y3 = y2

        x4 = x3
        y4 = marginal_costs[max_type]

        x5 = x4 + power_plant_p_nom['Bus'][max_type]
        y5 = y4

        fig = go.Figure()

        # Add shapes
        fig.add_shape(type="line",
            x0=x0, y0=y0, x1=x1, y1=y1,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text="Wind")
        )
        fig.add_shape(type="line",
            x0=x1, y0=y1, x1=x2, y1=y2,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x2, y0=y2, x1=x3, y1=y3,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=min_type)
        )
        fig.add_shape(type="line",
            x0=x3, y0=y3, x1=x4, y1=y4,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x4, y0=y4, x1=x5, y1=y5,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=max_type)
        )
        fig.update_shapes(dict(xref='x', yref='y'))

        fig.add_vline(x=network.loads_t.p['Bus load'][period], line_color="green", annotation_text="Load", annotation_position="bottom right")

        fig.update_xaxes(range=[0, network.generators.p_nom.sum() + 10], title_text='Quantity')
        fig.update_yaxes(range=[0, y5 + 10], title_text='Price')
        fig.update_layout(showlegend=False, margin=dict(t=20, b=20))

        return fig

    fig = demand_supply_diagram(0)

    return fig

app.run_server(mode='inline', port=8063, jupyter_height=600)

## Multiple time periods

apply above for multiple time periods...

In [3]:
import pypsa, numpy as np

# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

wind_profile = np.random.uniform(low=0, high=1, size=(24,))

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        dcc.Graph(id="graph2"),
        html.P("Select period:"),
        dcc.Slider(
            id="slider-period",
            min=0,
            max=23,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 24, 1)},
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Output("graph2", "figure"),
    Input("slider-period", "value")
)

def calc(period):

    # carbon emissions in tCO2/MWh
    carbon_emissions = {"Wind": 0, "Coal": 0.1, "Gas": 0.05}
    # operational costs in £/MWh
    operational_costs = {"Wind": 0, "Coal": 7, "Gas": 10}
    # marginal costs in £/MWh
    carbon_price = 80
    marginal_costs = {"Wind": carbon_emissions['Wind'] * carbon_price + operational_costs['Wind'],
                    "Coal": carbon_emissions['Coal'] * carbon_price + operational_costs['Coal'],
                    "Gas": carbon_emissions['Gas'] * carbon_price + operational_costs['Gas']}
    # power plant capacities (nominal powers in MW)
    wind_capacity = 2000
    power_plant_p_nom = {'Bus': {"Wind": wind_capacity, "Gas": 1000, "Coal": 4000}}

    network = pypsa.Network()
    # snapshots labelled by [0,1,2,3]
    country = "Bus"
    network.set_snapshots(range(24))
    network.add("Bus", country)

    # p_max_pu is variable for wind
    for tech in power_plant_p_nom[country]:
        network.add(
            "Generator",
            "{} {}".format(country, tech),
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
            p_max_pu=(wind_profile) if tech == "Wind" else 1)

    # load which varies over the snapshots
    network.add(
        "Load",
        "{} load".format(country),
        bus=country,
        p_set=[1500] * 24,
    )

    network.optimize(solver_name='highs')

    def demand_supply_diagram(period):

        # minimum marginal cost unit between gas and coal
        marginal_costs_ = marginal_costs
        del marginal_costs_['Wind']
        min_type = min(marginal_costs_, key=marginal_costs_.get)
        max_type = max(marginal_costs_, key=marginal_costs_.get)

        x0 = 0
        y0 = 0

        x1 = wind_capacity * network.generators_t.p_max_pu['Bus Wind'][period]
        y1 = 0

        x2 = x1
        y2 = marginal_costs[min_type]

        x3 = x2 + power_plant_p_nom['Bus'][min_type]
        y3 = y2

        x4 = x3
        y4 = marginal_costs[max_type]

        x5 = x4 + power_plant_p_nom['Bus'][max_type]
        y5 = y4

        fig = go.Figure()

        # Add shapes
        fig.add_shape(type="line",
            x0=x0, y0=y0, x1=x1, y1=y1,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text="Wind")
        )
        fig.add_shape(type="line",
            x0=x1, y0=y1, x1=x2, y1=y2,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x2, y0=y2, x1=x3, y1=y3,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=min_type)
        )
        fig.add_shape(type="line",
            x0=x3, y0=y3, x1=x4, y1=y4,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x4, y0=y4, x1=x5, y1=y5,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=max_type)
        )
        fig.update_shapes(dict(xref='x', yref='y'))

        fig.add_vline(x=network.loads_t.p['Bus load'][period], line_color="green", annotation_text="Load", annotation_position="bottom right")

        fig.update_xaxes(range=[0, network.generators.p_nom.sum() + 10], title_text='Quantity')
        fig.update_yaxes(range=[0, y5 + 10], title_text='Price')
        fig.update_layout(showlegend=False, margin=dict(t=20, b=20))

        return fig
    
    def generation_fig():
            
            fig = px.area(network.generators_t.p)#, facet_col=['Bus Coal', 'Bus Wind', 'Bus Gas'], facet_col_wrap=3, facet_row_spacing=0.05, facet_col_spacing=0.05)
            fig.update_yaxes(matches=None)
            fig.update_yaxes(title_text="Power (MW)")
            fig.update_xaxes(title_text="Period")
            fig.update_layout(showlegend=False, margin=dict(t=5, b=5))
    
            return fig

    fig = demand_supply_diagram(period)
    fig2 = generation_fig()

    return fig, fig2

app.run_server(mode='inline', port=8064, jupyter_height=1000)

## Energy storage

Energy storage and how it impacts here...

In [4]:
import pypsa, numpy as np

# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

wind_profile = np.random.uniform(low=0, high=1, size=(24,))
# load_profile = np.random.uniform(low=1000, high=1500, size=(24,))

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div(
    [
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph"),
        dcc.Graph(id="graph2"),
        html.P("Select period:"),
        dcc.Slider(
            id="slider-period",
            min=0,
            max=23,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 24, 1)},
        ),
    ]
)

@app.callback(
    Output("graph", "figure"),
    Output("graph2", "figure"),
    Input("slider-period", "value")
)

def calc(period):

    # carbon emissions in tCO2/MWh
    carbon_emissions = {"Wind": 0, "Coal": 0.1, "Gas": 0.05}
    # operational costs in £/MWh
    operational_costs = {"Wind": 0, "Coal": 7, "Gas": 10}
    # marginal costs in £/MWh
    carbon_price = 80
    marginal_costs = {"Wind": carbon_emissions['Wind'] * carbon_price + operational_costs['Wind'],
                    "Coal": carbon_emissions['Coal'] * carbon_price + operational_costs['Coal'],
                    "Gas": carbon_emissions['Gas'] * carbon_price + operational_costs['Gas']}
    # power plant capacities (nominal powers in MW)
    wind_capacity = 2000
    power_plant_p_nom = {'Bus': {"Wind": wind_capacity, "Gas": 1000, "Coal": 4000}}

    network = pypsa.Network()
    # snapshots labelled by [0,1,2,3]
    country = "Bus"
    network.set_snapshots(range(24))
    network.add("Bus", country)

    # p_max_pu is variable for wind
    for tech in power_plant_p_nom[country]:
        network.add(
            "Generator",
            tech,
            bus=country,
            p_nom=power_plant_p_nom[country][tech],
            marginal_cost=marginal_costs[tech],
            p_max_pu=(wind_profile) if tech == "Wind" else 1)

    # load which varies over the snapshots
    network.add(
        "Load",
        "{} load".format(country),
        bus=country,
        p_set=[1000] * 24,
    )

    network.add(
        "StorageUnit",
        "Storage",
        bus="Bus",
        marginal_cost=0,
        inflow=0,
        p_nom_extendable=False,
        capital_cost=10,
        p_nom=2000,
        efficiency_dispatch=0.5,
        cyclic_state_of_charge=False,
        state_of_charge_initial=0,
    )


    network.optimize(solver_name='highs')

    def demand_supply_diagram(period):

        # minimum marginal cost unit between gas and coal
        marginal_costs_ = marginal_costs
        del marginal_costs_['Wind']
        min_type = min(marginal_costs_, key=marginal_costs_.get)
        max_type = max(marginal_costs_, key=marginal_costs_.get)

        x0 = 0
        y0 = 0

        x1 = wind_capacity * network.generators_t.p_max_pu['Wind'][period]
        y1 = 0

        x2 = x1
        y2 = marginal_costs[min_type]

        x3 = x2 + power_plant_p_nom['Bus'][min_type]
        y3 = y2

        x4 = x3
        y4 = marginal_costs[max_type]

        x5 = x4 + power_plant_p_nom['Bus'][max_type]
        y5 = y4

        fig = go.Figure()

        # Add shapes
        fig.add_shape(type="line",
            x0=x0, y0=y0, x1=x1, y1=y1,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text="Wind")
        )
        fig.add_shape(type="line",
            x0=x1, y0=y1, x1=x2, y1=y2,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x2, y0=y2, x1=x3, y1=y3,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=min_type)
        )
        fig.add_shape(type="line",
            x0=x3, y0=y3, x1=x4, y1=y4,
            line=dict(color="RoyalBlue",width=3)
        )
        fig.add_shape(type="line",
            x0=x4, y0=y4, x1=x5, y1=y5,
            line=dict(color="RoyalBlue",width=3),
            label=dict(text=max_type)
        )
        fig.update_traces(name='consumer_surplus_text')
        fig.update_shapes(dict(xref='x', yref='y'))


        fig.add_vline(x=network.loads_t.p['Bus load'][period], line_color="green", annotation_text="Load", annotation_position="bottom right")

        fig.update_xaxes(range=[0, network.generators.p_nom.sum() + 10], title_text='Quantity')
        fig.update_yaxes(range=[0, y5 + 10], title_text='Price')
        fig.update_layout(showlegend=False, margin=dict(t=20, b=20))
        

        return fig
    
    def generation_fig():
            
            fig = px.area(network.generators_t.p)#, facet_col=['Bus Coal', 'Bus Wind', 'Bus Gas'], facet_col_wrap=3, facet_row_spacing=0.05, facet_col_spacing=0.05)
            fig.update_yaxes(matches=None)
            fig.update_yaxes(title_text="Power (MW)")
            fig.update_xaxes(title_text="Period")
            fig.update_layout(showlegend=False, margin=dict(t=20, b=5))

            fig_storage = px.line(network.storage_units_t.p)
            fig_storage.update_traces(line_color='black', line_dash="dash")
            fig_storage.update_layout(showlegend=False, margin=dict(t=5, b=5))
            fig = go.Figure(data=fig.data + fig_storage.data, layout = fig.layout)
    
            return fig

    fig = demand_supply_diagram(period)
    fig2 = generation_fig()

    return fig, fig2

app.run_server(mode='inline', port=8065, jupyter_height=1000)

## Example for Great Britain

![GB](2016_generation_dispatch.png)

## Including technical constraints

The example above assumed that you could dispatch each generator at any level between 0 and its capacity. In reality, this is not the case. Conventional generators (coal, gas, etc.), in particular, generally have many constraints on their operations. Once important constraint is that they generally cannot increase or decrease their production very quickly - it takes time to make large adjustments to output, similarly to how a car cannot go from 0 to 200mph instantaneously. These constraints are called ramping constraints. They limit the amount of generation possible in a particular period to be within a given range of the generation in the previous period. Generators that are very flexible (e.g., newer gas-fired generators) can ramp quickly. Generators that are inflexible (e.g. especially older nuclear power plants) cannot ramp quickly.

In addition to ramping constraints, conventional generators generally have minimum run levels. They cannot produce a positive amount below these levels. A 1GW generator might, for instance have a minimum run level of 50%, which means that it can only either generate nothing, or generate between 0.5 and 1GW. Generators therefore do not only decide how much to generate, but also whether they will generate at all. To make this more complicated, it generally takes time to start up a generator, so if a generator is turned off, it may not be able to generate anything in the immediate future. Generators might also have minimum up and down times, such that when they turn off they have to remain off for a number of periods, and when they are on, they have to remain on for a number of time periods. Decisions about being on or off are called unit commitment decisions. They are binary - units are either on (“online”) or off (“offline”). Economic dispatch problems with unit commitment constraints are sometimes called unit commitment problems.

## Markets in the real world

The above models are representations of markets. In the real world, depending on the type of market (as we will see next week), there might be a market operator which on a regular (e.g., hourly) basis solves problems similar to those above, but with many more generators and time periods, and generally using bids that generators/consumers have declared instead of their actual costs and benefits. This market operator then instructs the generators and consumers what to do in every period, with penalties for deviating from these instruction. In other markets, such as the UK, there is no centralised dispatch - generators make their own decisions. However, if we assume perfect competition, we would get to the same solution, so large-scale versions of the
models above can still be used by industry or by researchers to predict what will happen.